# Parte 2

Projeto Final do Turing de NLP para os Trainees - Parte 2

Essa parte do projeto se trata de aplicarmos um modelo baseado em um outro tipo de análise em NLP que é a aplicação com um método léxico, nesse caso sendo o SentimentIntensistyAnalyzer da biblioteca vaderSentiment_

In [1]:
#imports das bibliotecas para o nosso código
import pandas as pd
import nltk
from bs4 import BeautifulSoup
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [2]:
#ler o dataset
df = pd.read_csv("IMDB Dataset.csv")

In [3]:
def strip_html_tags(texto):
    soup = BeautifulSoup(texto, "html.parser")
    texto_tratado = soup.get_text(separator=" ")
    return texto_tratado

In [4]:
#aplicar a função para retirar as tags de html
df["review_tratado"] = df["review"].apply(lambda x: strip_html_tags(x))

## Aplicação do método léxico
* A aplicação do método polarity_scores retorna alguns valores sendo o que vai mais importar para gente o de "compound"
* Com esse valor, vamos transformar em "positive" tudo que for maior que 0 e "negative" tudo que for menor que 0
* A partir disso, vamos calcular as mesmas métricas calculadas na Parte 1
    * Acurácia
    * Matriz de confusão 
    * f1 score

In [5]:
#instanciando o modelo
analyser = SentimentIntensityAnalyzer()

In [6]:
analyser.polarity_scores(df["review_tratado"][0])

{'neg': 0.181, 'neu': 0.754, 'pos': 0.065, 'compound': -0.9916}

In [7]:
analyser.polarity_scores(df["review_tratado"][0])["compound"]

-0.9916

In [8]:
#aplicando o método de polarity_scores para o dataset inteiro e guardando na coluna "compound"
df["compound"] = df["review_tratado"].apply(lambda x: analyser.polarity_scores(x)["compound"])

In [9]:
#transformando em "positive" ou "negative" de acordo com o valor do "compound"
df["compound"] = df["compound"].apply(lambda x: "positive" if x>0 else "negative")

In [10]:
df["compound"].value_counts()

positive    33211
negative    16789
Name: compound, dtype: int64

In [11]:
#imports para a avaliação desse método
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

In [12]:
#calculando a acurácia
ACC = accuracy_score(df["sentiment"], df["compound"])
ACC

0.6983

In [13]:
#matriz de confusão
print(confusion_matrix(df['sentiment'], df['compound']))

[[13352 11648]
 [ 3437 21563]]


In [14]:
#outras métricas incluindo o f1 score
print(classification_report(df['sentiment'], df['compound']))

              precision    recall  f1-score   support

    negative       0.80      0.53      0.64     25000
    positive       0.65      0.86      0.74     25000

    accuracy                           0.70     50000
   macro avg       0.72      0.70      0.69     50000
weighted avg       0.72      0.70      0.69     50000



# Conclusão
Diferentemente dos outros modelos testados na parte 1, esse apresentou problemas um pouco mais preocupantes dada a sua avaliação. Tanto a variação de precision e de recall do modelo com "positive" e "negative" e também um número bem alto de valores de Falso Negative mostram que o nosso modelo está pior que os de antes. Mas mesmo assim, é bem interessante ver a aplicação de um método léxico que utiliza pouco pré-processamento (no caso, apenas retiramos as tags de html) e pensando nisso, atingiu um resultado satisfatório.

Gostei muito de entender um pouco mais sobre um tipo de método diferente que, por exemplo, considera se é upper ou lower case a palavra para predizer o sentimento da mensagem. Este é um traço que eu considero mais característica humana e pude assim me interessar ainda mais pela área de NLP.